In [1]:
import pandas as pd
# baic transformer Decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun
import numpy as np

main_df = pd.read_csv('adult.csv')
main_df.head()
DEVICE = 'cuda'

/home/tsou/.conda/envs/GNN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def POOL_preprocess(df):
    '''
    input the original dataframe, output the dataframe after preprocessing,
    change the numerical columns to categorical columns by qcut and cut
    then apply label encoding to all columns
    
    '''
    df = df.copy()
    CAT = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
    NUM = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
    # qcut on numerical columns
    for column in NUM:
        if column in ['educational-num','capital-gain','capital-loss','hours-per-week']:
            df[column] = pd.cut(df[column], 100)
        else:
            df[column] = pd.cut(df[column], 100)
    # make income column binary
    df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)

    # lable encoding categorical columns
    from sklearn.preprocessing import LabelEncoder
    lb = LabelEncoder()
    df = df.apply(lambda x: lb.fit_transform(x))

    # make all catagory in every column unique

    # 迴圈處理多個欄位
    offset = 0
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x + offset)
        offset += df[column].nunique()
    
    return df
tmp = POOL_preprocess(main_df)
tmp.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,8,78,97,164,185,199,209,220,225,229,230,253,341,437,440
1,21,78,88,174,187,197,207,217,227,229,230,253,351,437,440
2,11,76,104,170,190,197,213,217,227,229,230,253,341,437,441
3,27,78,93,178,188,197,209,217,225,229,237,253,341,437,441
4,1,74,89,178,188,199,202,220,227,228,230,253,331,437,440


In [3]:
tmp.describe()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,21.643585,77.870439,94.497113,173.288420,188.078089,197.618750,208.577700,218.443287,226.668052,228.668482,230.626858,254.102596,341.396728,434.749355,440.239282
std,13.710510,1.464234,7.118204,3.874492,2.570973,1.507703,4.230509,1.602151,0.845986,0.470764,2.670115,5.196314,12.295271,7.775343,0.426649
min,0.000000,74.000000,83.000000,163.000000,179.000000,195.000000,202.000000,217.000000,223.000000,228.000000,230.000000,253.000000,302.000000,398.000000,440.000000
25%,11.000000,78.000000,90.000000,172.000000,187.000000,197.000000,205.000000,217.000000,227.000000,228.000000,230.000000,253.000000,341.000000,437.000000,440.000000
50%,20.000000,78.000000,94.000000,174.000000,188.000000,197.000000,209.000000,218.000000,227.000000,229.000000,230.000000,253.000000,341.000000,437.000000,440.000000
75%,31.000000,78.000000,98.000000,175.000000,190.000000,199.000000,212.000000,220.000000,227.000000,229.000000,230.000000,253.000000,346.000000,437.000000,440.000000
max,73.000000,82.000000,162.000000,178.000000,194.000000,201.000000,216.000000,222.000000,227.000000,229.000000,252.000000,301.000000,397.000000,439.000000,441.000000


In [4]:
train_size = 4*48842//5
test_size = 48842//5
train_pool = main_df[test_size:]
test_pool = main_df[:test_size]
print('total data num:' , main_df.shape[0])
print('trian data num:' , train_pool.shape[0])
print('test data num:' , test_pool.shape[0])

total data num: 48842
trian data num: 39074
test data num: 9768


In [5]:
# notations
#   node: number of all nodes = L + S + C + F
#   L: number of lable nodes
#   S: number of sample nodes
#   C: number of catagory nodes
#   F: number of field(column) nodes
#   hidden: number of hidden representation

# data size = (node, hidden)
# mask size = (node, node - L) without lable nodes
#             for each node, real mask = cat[mask,(node,L)] = (node, node)
#             cannot see it's label node

# use nn.transformerDecoder(data,mask) to get the output
# use the above output as input of MLP to predict the lable   

In [6]:
TARGET_POOL = POOL_preprocess(main_df)
LABEL_COLUMN = 'income'
# 
HIDDEN = 64

# cut feature and lable
FEATURE_POOL = TARGET_POOL.drop(LABEL_COLUMN, axis=1)
LABEL_POOL = TARGET_POOL[LABEL_COLUMN]

# trasform label into one-hot
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
LABEL_POOL = enc.fit_transform(LABEL_POOL.values.reshape(-1,1)).toarray()

print('feature pool shape:', FEATURE_POOL.shape)
print('label pool shape:', LABEL_POOL.shape)

# L: number of lable nodes
L = LABEL_POOL.shape[1]

# S: number of sample nodes
S = FEATURE_POOL.shape[0]

# C: number of catagory nodes
C = FEATURE_POOL.apply(lambda x: x.nunique()).sum() # total_unique_labels
C_POOL = list(set(FEATURE_POOL.values.flatten()))

# F: number of field(column) nodes
F = FEATURE_POOL.shape[1]

print('L:', L)
print('S:', S)
print('C:', C)
print('F:', F)
print('total', L+S+C+F)
print(C_POOL)
print(len(C_POOL))


feature pool shape: (48842, 14)
label pool shape: (48842, 2)
L: 2
S: 48842
C: 440
F: 14
total 49298
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,

In [7]:
LABEL_POOL

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [8]:
# caculate masking
masks = {}

# label to sample 
edge_x_L2S = []
edge_y_L2S = []
label_ids = TARGET_POOL[LABEL_COLUMN].unique()
for i, value_df in enumerate(TARGET_POOL[LABEL_COLUMN]):
    for j, value_label in enumerate(label_ids):
        if value_label == value_df:
            edge_x_L2S.append(i)
            edge_y_L2S.append(j)
# make value on edge as 1
value_on_edge = [1]*len(edge_x_L2S)

indices = torch.tensor([edge_x_L2S, edge_y_L2S])
values = torch.tensor(value_on_edge)
size = torch.Size([S, L])
masks['L2S'] = torch.sparse_coo_tensor(indices, values, size)

# sample to catagory
edge_x_S2C = []
edge_y_S2C = []
tmp_df = TARGET_POOL.drop(LABEL_COLUMN, axis=1)
for i, value_df in enumerate(tmp_df.values):
    for j, value in enumerate(value_df):
        edge_x_S2C.append(value)
        edge_y_S2C.append(i)
# make value on edge as 1
value_on_edge = [1]*len(edge_x_S2C)
indices = torch.tensor([edge_x_S2C, edge_y_S2C])
values = torch.tensor(value_on_edge)
size = torch.Size([C, S])
masks['S2C'] = torch.sparse_coo_tensor(indices, values, size)


# catagory to field
edge_x_C2F = []
edge_y_C2F = []
unique_items = [(TARGET_POOL[column].unique()) for column in (TARGET_POOL.columns)]
for i in range(F):
    for j in (unique_items[i]):
        edge_x_C2F.append(i)
        edge_y_C2F.append(j)
# make value on edge as 1
value_on_edge = [1]*len(edge_x_C2F)
indices = torch.tensor([edge_x_C2F, edge_y_C2F])
values = torch.tensor(value_on_edge)
size = torch.Size([F, C])
masks['C2F'] = torch.sparse_coo_tensor(indices, values, size)

# to-do: unseen data issue


In [9]:
# make input tensor
# L
L_input = torch.eye(L).to(DEVICE)
print('L_input', L_input.type(), L_input.shape)
# S
S_input = torch.tensor(FEATURE_POOL.values).to(DEVICE)
print('S_input', S_input.type(), S_input.shape)
# C random init
C_input = torch.tensor(np.diag(C_POOL)).to(DEVICE)
print('C_input', C_input.type(), C_input.shape)
# F random init
F_input = torch.eye(F).to(DEVICE)
print('F_input', F_input.type(), F_input.shape)
print(L_input.type())
# 

L_input torch.cuda.FloatTensor torch.Size([2, 2])
S_input torch.cuda.LongTensor torch.Size([48842, 14])
C_input torch.cuda.LongTensor torch.Size([440, 440])
F_input torch.cuda.FloatTensor torch.Size([14, 14])
torch.cuda.FloatTensor


In [10]:
from torch import Tensor
from typing import Optional, Any, Union, Callable

class CustomTransformerDecoderLayer(nn.TransformerDecoderLayer):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu'):
        super().__init__(d_model, nhead, dim_feedforward, dropout, activation)
        # remove defined modules
        delattr(self, 'self_attn')
        delattr(self, 'norm1')
        delattr(self, 'dropout1')
    
    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        x = tgt
        if self.norm_first:
            # x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask)
            x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask)
            x = x + self._ff_block(self.norm3(x))
        else:
            # x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
            x = self.norm3(x + self._ff_block(x))

        return x


In [11]:
# baic transformer decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun

class TransformerDecoderModel(nn.Module):
    def __init__(self, nodes_num, num_layers, embedding_dim, hidden_dim):
        super(TransformerDecoderModel, self).__init__()

        L_dim, S_dim, C_dim, F_dim = nodes_num
        
        # 目前b卡在embedding的怎麼用
        # Catagory_embedding => 數值類Qcut後用linear來做embedding, 類別用nn.Embedding

        self.Lable_embedding = nn.Linear(L_dim, embedding_dim, dtype=torch.float)
        self.Sample_embedding = nn.Linear(S_dim, embedding_dim, dtype=torch.float)
        self.Catagory_embedding = nn.Linear(C_dim, embedding_dim, dtype=torch.float)
        self.Field_embedding = nn.Linear(F_dim, embedding_dim, dtype=torch.float)
        
        self.transformer_decoder = nn.TransformerDecoder(
            CustomTransformerDecoderLayer(embedding_dim,  nhead = 8 ),
            num_layers
        )
        
        # downstream task
        self.MLP = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim//2),
            nn.ReLU(),
            nn.Linear(embedding_dim//2, embedding_dim//3),
            nn.ReLU(),
            nn.Linear(embedding_dim//3, 2),
            nn.Softmax(dim=1)
        )


    def forward(self, L_input, S_input, C_input, F_input, masks):
        L_embedded = self.Lable_embedding(L_input.float())
        S_embedded = self.Sample_embedding(S_input.float())
        C_embedded = self.Catagory_embedding(C_input.float())
        F_embedded = self.Field_embedding(F_input.float())
        
        for mask in masks.keys():
            masks[mask] = masks[mask].float().to_dense().to(DEVICE)
        
        # propagate steps: L→S→C→F→C→S→L
        # more steps more menory usage
        PROPAGATE_STEPS = 2
        for i in range(PROPAGATE_STEPS):
            S_embedded = self.transformer_decoder(S_embedded,L_embedded, 
                                                memory_mask = masks['L2S']) 
            C_embedded = self.transformer_decoder(C_embedded,S_embedded,
                                                memory_mask = masks['S2C'])
            F_embedded = self.transformer_decoder(F_embedded,C_embedded,
                                                memory_mask = masks['C2F'])
            C_embedded = self.transformer_decoder(C_embedded,F_embedded,
                                                memory_mask = masks['C2F'].transpose(0, 1))
            S_embedded = self.transformer_decoder(S_embedded,C_embedded,
                                                memory_mask = masks['S2C'].transpose(0, 1))
            L_embedded = self.transformer_decoder(L_embedded,S_embedded, 
                                                memory_mask = masks['L2S'].transpose(0, 1))
        
        # S_embedded = self.transformer_decoder(S_embedded,L_embedded, memory_key_padding_mask   = masks['L2S'].float().to(DEVICE))  # 使用 TransformerDecoder 編碼
        output = self.MLP(S_embedded)
        return output
        # return encoded.permute(1, 0, 2)  # 改變 tensor 的維度順序回來

# 測試模型
num_layers = 1  # TransformerDecoder 的層數
embedding_dim = 128  # 嵌入維度
hidden_dim = 64  # TransformerDecoderLayer 的隱藏層維度

nodes_num = (L_input.size(1), S_input.size(1), C_input.size(1), F_input.size(1))
print('nodes_num', nodes_num)
model = TransformerDecoderModel(nodes_num, num_layers, embedding_dim, hidden_dim).to(DEVICE)
output = model(L_input, S_input, C_input, F_input, masks)

print("模型輸出的大小:", output.shape)

nodes_num (2, 14, 440, 14)


模型輸出的大小: torch.Size([48842, 2])


In [15]:
output_label = torch.argmax(output, dim=1)
output_label.sum()

tensor(48839, device='cuda:0')

In [13]:
# caculate accuracy
acc = torch.sum(output_label == torch.argmax(torch.tensor(LABEL_POOL).to(DEVICE), dim=1)).item() / len(output_label)
acc

0.23926129151140413